In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display
pd.set_option('display.max_columns', None)


import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn import linear_model, metrics
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import explained_variance_score


# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

# Exploratory Data Analysis

In [ ]:
#To Load Data , specify appropriate file path 
basePath = "" #Path where file is located
df = pd.read_csv(basePath + "train.csv")
df.shape

In [ ]:
df.info()

### Data Cleansing and preparation


In [ ]:
# Remove columns having more than 70% null values as seen from info above
df.drop(['Alley','PoolQC','Fence','MiscFeature'],axis =1 , inplace = True)


In [ ]:
df.describe()

In [ ]:
#handle null values for important columns
#For LotFrontage, mean and median are close plus there are few outliars, lets replace nulls with median
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())
#For MasVnrArea , missing values mean , MasVnrType was NA , lets replace with 0
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
#For GarageYrBlt , missing values mean , GarageType was NA , lets replace with YearBuilt since no garage was built
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df['YearBuilt'])
# For BsmtFinType1 means no basement
#Similarly from data dictionary, we know that missing values are NA or None , lets replace them like this for now
df['GarageFinish'] = df['GarageFinish'].fillna('NA')
df['GarageType'] = df['GarageType'].fillna('NA')
df['KitchenQual'] = df['KitchenQual'].fillna('NA')
df['BsmtExposure'] = df['BsmtExposure'].fillna('NA')
df['BsmtQual'] = df['BsmtQual'].fillna('NA')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('NA')
df['MasVnrType'] = df['MasVnrType'].fillna('None')
df['FireplaceQu'] = df['FireplaceQu'].fillna('NA')

In [ ]:
# Certain columns may have values that are biased towards certain category such that dominant category has > 95% weight. 
#Lets remove them.
for each_column in df.columns:
    if df[each_column].dtypes == 'object':
        val_cnt_df =  df[each_column].value_counts(normalize = True).to_frame()
        print(val_cnt_df)
        print('-------------------------------------')
        if val_cnt_df.iloc[0,0] > 0.95:
            df.drop(each_column,axis =1 , inplace = True)
            print('Deleted column : ' , each_column)
            print('-------------------------------------') 
          


In [ ]:
# Find correlations with respect to response variable SalePrice
SalesPriceCorr = df.corr()['SalePrice'].to_frame().sort_values(by='SalePrice',ascending = False)
SalesPriceCorr

In [ ]:
# Remove columns having low correlation with SalePrice , making sure that useful columns are retained ,
# while doing this we are more liberal towards negative correlations since we want to buy at lower prices
SalesPriceCorr['Cols_Remove'] = np.where(((SalesPriceCorr['SalePrice'] < 0.25) & (SalesPriceCorr['SalePrice'] > -0.05)) ,SalesPriceCorr.index,'KeepThisCol')
for eachCol in SalesPriceCorr['Cols_Remove']:
    if eachCol != 'KeepThisCol':
        df.drop(eachCol,axis =1 , inplace = True)
        print(eachCol)
SalesPriceCorr = df.corr()['SalePrice'].to_frame()
SalesPriceCorr

#### Univariate Analysis for Numeric Variables

In [ ]:
df_num = df.select_dtypes(include=[np.number]).columns
for each_numeric_column in df_num:
    sns.boxplot(y = each_numeric_column , data = df)
    plt.show()

In [ ]:
for eachCol in df.select_dtypes(include=[np.number]).columns:
    sns.regplot(y = 'SalePrice' , x = eachCol , data = df)
    plt.show()

In [ ]:
#Combine areas of 2 floors as second floor area is 0 when the floor doesnt exist and remove separate columns for 2 areas
df['TotFlrSF'] = df['1stFlrSF'] + df['2ndFlrSF']
df.drop(['1stFlrSF','2ndFlrSF'] , axis =1 , inplace = True)

In [ ]:
# 'LotFrontage', 'Lotarea' have low correlation with target and also they have outliars, 
# Also from domain, we know that they are related
# we will categorize these columns first and see the impact


In [ ]:
# Generic Funtion for binning 
def convert_numeric_column_to_categorical_analyze(numeric_column):
    q=[0,.25, .5, .75,1]
    labels = ['q1','q2','q3','q4']
    df['binned_' + numeric_column] = pd.qcut(df[numeric_column], q=q, labels=labels)
    print(numeric_column)
    df.drop(numeric_column, axis =1 , inplace = True)
    df['binned_' + numeric_column] = df['binned_' + numeric_column].astype('object')

In [ ]:
for column in ['LotFrontage', 'LotArea']:
    convert_numeric_column_to_categorical_analyze(column)

#### Analysis of categorical variables

In [ ]:
# Overlay median , q1 and q3 values for SalePrice over boxplot for instant readymade insights
for each_column in df.columns:
    if df[each_column].dtypes == 'object':
        grouped = df.loc[:,[each_column, 'SalePrice']].groupby(each_column) .median().sort_values(by='SalePrice')
        ax = sns.boxplot(x = each_column , y = 'SalePrice' ,data = df , order = grouped.index)
        sns.swarmplot(x = each_column , y = 'SalePrice', data=df, zorder=.5, order = grouped.index)
        ax.axhline(df['SalePrice'].median(),linewidth=1,color='b')
        ax.axhline(df['SalePrice'].quantile(0.75),linewidth=1,color='r')
        ax.axhline(df['SalePrice'].quantile(0.25),linewidth=1,color='g')
        ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
        plt.show()
        print('-----------------------------------------------------------------------------------')


In [ ]:
#SaleType though useful info. is not helpful in model building and determinant of SalePrice, drop it
df.drop('SaleType' , axis =1 , inplace = True)

In [ ]:
#Exterior1st and Exterior2nd also have many categories and some categories have lesser weights, 
#lets first combine these categories
# And then combine two columns to create a single exterior column and handle nulls
df['Exterior1st'] = df['Exterior1st'].map({'VinylSd' : 'VinylSd' , 'HdBoard':'HdBoard' ,'MetalSd' :'MetalSd','Wd Sdng':'Wd Sdng',
                                          'Plywood' : 'Modified1' ,'CemntBd': 'Modified1','BrkFace': 'Modified1',
                                           'WdShing' : 'Modified2', 'Stucco': 'Modified2' ,'AsbShng': 'Modified2','Stone': 'Modified2'
                                           ,'BrkComm' : 'Modified2','ImStucc': 'Modified2','CBlock': 'Modified2', 'AsphShn':'Modified2'
                                          })
df['Exterior2nd'] = df['Exterior2nd'].map({'VinylSd' : 'VinylSd' , 'HdBoard':'HdBoard' ,'MetalSd' :'MetalSd','Wd Sdng':'Wd Sdng',
                                          'Plywood' : 'Modified1' ,'CemntBd': 'Modified1','BrkFace': 'Modified1', 
                                           'WdShing' : 'Modified2', 'Stucco': 'Modified2' ,'AsbShng': 'Modified2','Stone': 'Modified2'
                                           ,'Brk Cmn' : 'Modified2','ImStucc': 'Modified2','CBlock': 'Modified2', 'AsphShn':'Modified2',
                                           'Other' : 'Modified2'
                                          })

df['Exterior'] = np.where(df['Exterior1st'] == df['Exterior2nd'],df['Exterior1st'],df['Exterior1st'] + "_" + df['Exterior2nd'])

df['Exterior'] = df['Exterior'].fillna('Modified2')
df.drop(['Exterior2nd','Exterior1st'],axis =1 , inplace = True)

# Further Reduce categories in Exterior and handle nulls
df['Exterior'] = df['Exterior'].map({'VinylSd' : 'VinylSd' , 'HdBoard':'HdBoard' ,'MetalSd' :'MetalSd','Wd Sdng':'Wd Sdng',
                                          'Plywood' : 'Modified1' ,'Modified2' : 'Modified2'})
df['Exterior'] = df['Exterior'].fillna('Modified3')

In [ ]:
# For Neighborhood , apply frequency encoding as there are too many categories and small no. of records in each category
fe = df.groupby("Neighborhood").size()
fe_ = fe/len(df)
df["Neighborhood"] = df["Neighborhood"].map(fe_).round(2)
df['SalePrice'].corr(df["Neighborhood"])

In [ ]:
#Similar exercise for Condition1
fe = df.groupby("Condition1").size()
fe_ = fe/len(df)
df["Condition1"] = df["Condition1"].map(fe_).round(2)
df['SalePrice'].corr(df["Condition1"])

In [ ]:
#Bivariate analysis of categorical columns
lst_categories = df.select_dtypes(include=['object']).columns
for i, eachCategory1 in enumerate(lst_categories):
    for j, eachCategory2 in enumerate(lst_categories):
        if j<=i :
            continue
        print('------------------------------------------------------')
        ax=df.boxplot(column='SalePrice' ,by=[eachCategory1,eachCategory2] , rot = 90)
        ax.axhline(df['SalePrice'].median(),linewidth=1,color='b')
        ax.axhline(df['SalePrice'].quantile(0.75),linewidth=1,color='r')
        ax.axhline(df['SalePrice'].quantile(0.25),linewidth=1,color='g')
        plt.show() 
        print('------------------------------------------------------')


In [ ]:
#For Some categorical variables especially related to quality looking at the box plot plus data dictionary,
#We know that there is ordinality. Lets convert them to numeric maintaining the order.
df['ExterQual'] = df['ExterQual'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df['BsmtQual'] = df['BsmtQual'].map({'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
df['BsmtCond'] = df['BsmtCond'].map({'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
df['BsmtExposure'] = df['BsmtExposure'].map({'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4})
df['HeatingQC'] = df['HeatingQC'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df['KitchenQual'] = df['KitchenQual'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df['GarageFinish'] = df['GarageFinish'].map({'NA':0,'Unf':1,'RFn':2,'Fin':3})

In [ ]:
#For now, to make bi/multivariate analysis easier , lets convert categorical objects into numeric objects with label encoding
# Being linear regression, this has side effect of introducing ordinality, however this is just intermediate step and 
#before regression we will one-hot/dummy encode them
#Create a list of cetegorical features and maintain it live for ease of handling dummy encoding
cat_cols_later_dummy_encoding = []
for each_column in df.columns:
    if df[each_column].dtypes == 'object':
        df[each_column] = df[each_column].astype('category')
        df[each_column] = df[each_column].cat.codes
        cat_cols_later_dummy_encoding.append(each_column)


In [ ]:
#As seen from scatterplot, some numeric variables are also categorical . 
#Lets see if if they are indeed ordinal and/or if there is need to create dummies from them when no. of categories is small.
numericColsWhichCategorical = ['FullBath','HalfBath','KitchenAbvGr','Fireplaces','GarageCars']
for each_column in numericColsWhichCategorical:
    grouped = df.loc[:,[each_column, 'SalePrice']].groupby(each_column) .median().sort_values(by='SalePrice')
    ax = sns.boxplot(x = each_column , y = 'SalePrice' ,data = df , order = grouped.index)
    sns.swarmplot(x = each_column , y = 'SalePrice', data=df, zorder=.5, order = grouped.index)
    ax.axhline(df['SalePrice'].median(),linewidth=1,color='b')
    ax.axhline(df['SalePrice'].quantile(0.75),linewidth=1,color='r')
    ax.axhline(df['SalePrice'].quantile(0.25),linewidth=1,color='g')
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    plt.show()
    print('-----------------------------------------------------------------------------------')


In [ ]:
# Find correlations with respect to response variable SalePrice 
# And remove correlations with low correlations with a bit stricter criteria compared to first pass
# as this will help us build a fine tuned yet simple model
SalesPriceCorr = df.corr()['SalePrice'].to_frame()
SalesPriceCorr['Cols_Remove'] = np.where(((SalesPriceCorr['SalePrice'] < 0.4) & (SalesPriceCorr['SalePrice'] > -0.2)) ,SalesPriceCorr.index,'KeepThisCol')
for eachCol in SalesPriceCorr['Cols_Remove']:
    if eachCol != 'KeepThisCol':
        df.drop(eachCol,axis =1 , inplace = True)
        if eachCol in numericColsWhichCategorical:
            numericColsWhichCategorical.remove(eachCol)
        if eachCol in cat_cols_later_dummy_encoding:
            cat_cols_later_dummy_encoding.remove(eachCol)
        print(eachCol)
SalesPriceCorr = df.corr()['SalePrice'].to_frame()
SalesPriceCorr

In [ ]:
# Correlation Matrix for numeric variables

In [ ]:
plt.figure(figsize= (26,30))
sns.heatmap(df.corr(),annot = True)

In [ ]:
#Observations:
# Garage  Type and Neighbourhood have in general weaker correlation with other columns
#OveallQuality has good correlation with most of the columns.


In [ ]:
# YearBuilt and YearRemodAdd have strong +ve correlation with each other and with SalePrice, 
# lets create derived variable to make YearRemodAdd as 0 when there is None, that makes the correlation -ve. 
df['YearRemodAdd'] = np.where (df['YearBuilt'] == df['YearRemodAdd'],0,df['YearRemodAdd'])
    #df[['YearBuilt','YearRemodAdd']].max(axis =1)
print(df['SalePrice'].corr(df['YearRemodAdd']))
print(df['YearBuilt'].corr(df['YearRemodAdd']))

In [ ]:
#garage year and year build have strong correlation ,hence dropping former as it is less important than later
# and also has other correlated columns

df.drop(['GarageYrBlt','GarageCars'] , axis =1 , inplace = True)

In [ ]:
#Lets analyze VIF and build a model before dummy variable creation to understand more about multicollinearity 
vif = pd.DataFrame()
vif['Features'] = df.columns
vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# TotFlrSF has highest VIF and its derived
#lets drop 
df.drop(['TotFlrSF'] , axis =1 , inplace = True)



In [ ]:
#Lets build a raw model first to see at high level if we can further cleanse the data before dummy variable creation

In [ ]:
# Split into train and test/validation set assuming this will also be tested on separate test data
np.random.seed(0)
df_train,df_validation = train_test_split(df,train_size =0.8 , test_size = 0.2 , random_state = 100)

In [ ]:
# Apply standardization
scaler = MinMaxScaler()
allCols = df_train.columns.values.tolist()
df_train[allCols] = scaler.fit_transform(df_train[allCols])

In [ ]:
y_train_raw = df_train.pop('SalePrice')
X_train_raw = df_train

In [ ]:
#Intially use stats models to build a model 
# where in multicollinearity is handled to an extent and also p values are taken care of 
x_train_raw_lm = sm.add_constant(X_train_raw)
lr_raw = sm.OLS(y_train_raw,x_train_raw_lm).fit()

In [ ]:
lr_raw.summary()

In [ ]:
#Use RFE to reduce/prioritize the features
lm_raw_rfe = LinearRegression()
lm_raw_rfe.fit(X_train_raw, y_train_raw)

rfe_raw = RFE(lm_raw_rfe, 15)             # running RFE
rfe_raw = rfe_raw.fit(X_train_raw, y_train_raw)

In [ ]:
list(zip(X_train_raw.columns,rfe_raw.support_,rfe_raw.ranking_))

In [ ]:
#Build a model based on rfe guidance
x_train_raw_rfe = X_train_raw[X_train_raw.columns[rfe_raw.support_]]
x_train_raw_rfe_lm = sm.add_constant(x_train_raw_rfe)


In [ ]:
lr_raw_rfe = sm.OLS(y_train_raw, x_train_raw_rfe_lm).fit()
lr_raw_rfe.summary()

In [ ]:
# Get the p values in shape first by dropping non-significant features 

In [ ]:
x_train_3 = x_train_raw_rfe.drop('FullBath', axis=1) 
x_train_3_lm = sm.add_constant(x_train_3)
lr_3 = sm.OLS(y_train_raw, x_train_3_lm).fit()
lr_3.summary()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = x_train_3.columns
vif['VIF'] = [variance_inflation_factor(x_train_3.values, i) for i in range(x_train_3.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# With a reasonable raw model with optimal variables,create dummy variables to proceed further in final model building

In [ ]:
my_lst = list(x_train_3.columns)
my_lst.append('SalePrice')

In [ ]:
df = df[my_lst]
df.info()

Dummy Variable Creation

In [ ]:
#Dummy encode nominal categorical varaibles 
for each_column in cat_cols_later_dummy_encoding:
    if each_column in df:
        dummyCol = pd.get_dummies(df[each_column],prefix = each_column ,drop_first = True)
        df = pd.concat([df,dummyCol], axis = 1)
        df.drop(each_column,axis =1 , inplace = True)
for each_column in numericColsWhichCategorical:
    if each_column in df:
        dummyCol = pd.get_dummies(df[each_column],prefix = each_column ,drop_first = True)
        df = pd.concat([df,dummyCol], axis = 1)
        df.drop(each_column,axis =1 , inplace = True)

In [ ]:
df.info()

# Data split and scaling of training data

In [ ]:
#Split data in training ,validation and testing/validation sets , calling it validation as file given was for training data
np.random.seed(0)
df_train,df_validation = train_test_split(df,train_size =0.8 , test_size = 0.2 , random_state = 100)


In [ ]:
scaler = MinMaxScaler()
allCols = df_train.columns.values.tolist()
df_train[allCols] = scaler.fit_transform(df_train[allCols])
df_train.head()

In [ ]:
y_train = df_train.pop('SalePrice')
X_train = df_train


# Model Building 

In [ ]:
# Build a all variable linear model using statsmodel to make fine tuning easier

In [ ]:
x_train_lm = sm.add_constant(X_train)
lr_all = sm.OLS(y_train,x_train_lm).fit()


In [ ]:
lr_all.summary()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Gradually drop variables to arrive at optimal model that does not overfit and
# has reasonable multicollinearity

In [ ]:
x_train_6 = X_train.drop('Fireplaces_1', axis=1) 
x_train_6_lm = sm.add_constant(x_train_6)
lr_6 = sm.OLS(y_train, x_train_6_lm).fit()
lr_6.summary()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
x_train_6 = x_train_6.drop('BsmtQual', axis=1) 
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
x_train_6 = x_train_6.drop('KitchenQual', axis=1) 
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
x_train_6 = x_train_6.drop('YearBuilt', axis=1) 
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
x_train_6 = x_train_6.drop('OverallQual', axis=1) 
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
x_train_6_lm = sm.add_constant(x_train_6)
lr_6 = sm.OLS(y_train, x_train_6_lm).fit()
lr_6.summary()

In [ ]:
x_train_6 = x_train_6.drop('ExterQual', axis=1) 
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
x_train_6_lm = sm.add_constant(x_train_6)
lr_6 = sm.OLS(y_train, x_train_6_lm).fit()
lr_6.summary()

In [ ]:
x_train_6 = x_train_6.drop('binned_LotArea_2', axis=1) 
x_train_6_lm = sm.add_constant(x_train_6)
lr_6 = sm.OLS(y_train, x_train_6_lm).fit()
lr_6.summary()


In [ ]:
x_train_6 = x_train_6.drop('binned_LotArea_1', axis=1) 
x_train_6_lm = sm.add_constant(x_train_6)
lr_6 = sm.OLS(y_train, x_train_6_lm).fit()
lr_6.summary()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
x_train_6 = x_train_6.drop('YearRemodAdd', axis=1) #['binned_LotArea_3','Fireplaces_2','Fireplaces_3']
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
x_train_6_lm = sm.add_constant(x_train_6)
lr_6 = sm.OLS(y_train, x_train_6_lm).fit()
lr_6.summary()

In [ ]:
x_train_6 = x_train_6.drop('binned_LotArea_3', axis=1)
x_train_6_lm = sm.add_constant(x_train_6)
lr_6 = sm.OLS(y_train, x_train_6_lm).fit()
lr_6.summary()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = x_train_6.columns
vif['VIF'] = [variance_inflation_factor(x_train_6.values, i) for i in range(x_train_6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
#So we have a stable model now, lets validate assumptions of linear regression

In [ ]:
y_train_pred = lr_6.predict(x_train_6_lm)
fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)  

In [ ]:
#look for patterns in residuals
sns.scatterplot(x_train_6.index,y_train - y_train_pred)
plt.show()

# Regularization and model evaluation and comparisons

In [ ]:
# Before regularization, Just creating same model using sklearn for easier comparisons
reg = LinearRegression() 
reg.fit(x_train_6,y_train)

In [ ]:
# Print the coefficients and intercept
print(reg.intercept_)
print(reg.coef_)

In [ ]:
#Predictions on validation data 
df_validation[allCols] = scaler.transform(df_validation[allCols])
y_validation = df_validation.pop('SalePrice')
X_validation = df_validation[x_train_6.columns]


In [ ]:
def model_effectiveness(y_train,y_pred_train,y_validation,y_pred_validation,explained_variance = 0):
    metric = []
    r2_train_lr = r2_score(y_train, y_pred_train)
    print(r2_train_lr)
    metric.append(r2_train_lr)

    r2_test_lr = r2_score(y_validation, y_pred_validation)
    print(r2_test_lr)
    metric.append(r2_test_lr)

    rss1_lr = np.sum(np.square(y_train - y_pred_train))
    print(rss1_lr)
    metric.append(rss1_lr)

    rss2_lr = np.sum(np.square(y_validation - y_pred_validation))
    print(rss2_lr)
    metric.append(rss2_lr)

    mse_train_lr = mean_squared_error(y_train, y_pred_train)
    print(mse_train_lr)
    metric.append(mse_train_lr**0.5)

    mse_test_lr = mean_squared_error(y_validation, y_pred_validation)
    print(mse_test_lr)
    metric.append(mse_test_lr**0.5)
    
    print(explained_variance)
    metric.append(explained_variance)
    return metric

In [ ]:
y_pred_train = reg.predict(x_train_6)
y_pred_validation = reg.predict(X_validation)
metricReg = model_effectiveness(y_train,y_pred_train,y_validation,y_pred_validation)


In [ ]:
#Lets plot results and see if predictions are reasonable
fig = plt.figure()
plt.scatter(y_validation, y_pred_validation)
fig.suptitle('y_validation vs y_pred_validation', fontsize = 20)              # Plot heading 
plt.xlabel('y_validation', fontsize = 18)                          # X-label
plt.ylabel('y_pred_validation', fontsize = 16)   

### Ridge and Lasso

In [ ]:
def runRegularization(estimator,scoring,folds):
    model_cv = GridSearchCV(estimator = estimator, 
                    param_grid = params, 
                    scoring= 'explained_variance',  
                    cv = folds, 
                    return_train_score=True,
                    verbose = 1) 
    return(model_cv)


In [ ]:
# list of alphas to tune - if value too high it will lead to underfitting, if it is too low, 
# it will not handle the overfitting
# use explained_variance as it is bounded
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}

ridge1 = Ridge()
ridge1_model_cv_explained_variance = runRegularization(ridge1,'explained_variance',5)
    
ridge1_model_cv_explained_variance.fit(x_train_6, y_train) 



In [ ]:
#Fitting Ridge model for alpha = 3 and printing coefficients which have been penalised
ridge1 = Ridge(alpha=ridge1_model_cv_explained_variance.best_params_['alpha'])
ridge1.fit(x_train_6, y_train)


In [ ]:
y_pred_train = ridge1.predict(x_train_6)
y_pred_validation = ridge1.predict(X_validation)

metricRidge1 = model_effectiveness(y_train,y_pred_train,y_validation,y_pred_validation,explained_variance_score(y_train,y_pred_train))

In [ ]:
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}

ridge2 = Ridge()
ridge2_model_cv_neg_mean_absolute_error = runRegularization(ridge2,'neg_mean_absolute_error',10)
    
ridge2_model_cv_neg_mean_absolute_error.fit(x_train_6, y_train) 

In [ ]:
#Build another ridge model with alpha double than previous model
ridge2 = Ridge(alpha=2*ridge2_model_cv_neg_mean_absolute_error.best_params_['alpha'])
ridge2.fit(x_train_6, y_train)

In [ ]:
y_pred_train = ridge2.predict(x_train_6)
y_pred_validation = ridge2.predict(X_validation)

metricridge2 = model_effectiveness(y_train,y_pred_train,y_validation,y_pred_validation,explained_variance_score(y_train,y_pred_train))

In [ ]:

params = {'alpha': [0.000008,0.0001,0.0006, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}
# cross validation
lasso1 = Lasso()
lasso1_model_cv_explained_variance = runRegularization(lasso1,'explained_variance',5)
    
lasso1_model_cv_explained_variance.fit(x_train_6, y_train)            


In [ ]:
lasso1 = Lasso(alpha=lasso1_model_cv_explained_variance.best_params_['alpha'])
lasso1.fit(x_train_6, y_train)

In [ ]:
y_pred_train = lasso1.predict(x_train_6)
y_pred_validation = lasso1.predict(X_validation)

metricLasso1 = model_effectiveness(y_train,y_pred_train,y_validation,y_pred_validation,explained_variance_score(y_train,y_pred_train))


In [ ]:
# Build a model with double alpha
lasso2 = Lasso(alpha=2*lasso1_model_cv_explained_variance.best_params_['alpha'])
lasso2.fit(x_train_6, y_train)

In [ ]:
y_pred_train = lasso2.predict(x_train_6)
y_pred_validation = lasso2.predict(X_validation)

metricLasso2 = model_effectiveness(y_train,y_pred_train,y_validation,y_pred_validation,explained_variance_score(y_train,y_pred_train))

In [ ]:
# Creating a table which contain all the metrics for
# linear model , regularized models and model using twice regularization alpha

lr_table = {'Metric': ['R2 Score (Train)','R2 Score (Test)','RSS (Train)','RSS (Test)',
                       'MSE (Train)','MSE (Test)','ExplainedVariance'], 
        'Linear Regression': metricReg
        }

lr_metric = pd.DataFrame(lr_table ,columns = ['Metric', 'Linear Regression'] )

rg1_metric = pd.Series(metricRidge1, name = 'Ridge1')
rg2_metric = pd.Series(metricridge2, name = 'Ridge2')
ls1_metric = pd.Series(metricLasso1, name = 'Lasso1')
ls2_metric = pd.Series(metricLasso2, name = 'Lasso2')


final_metric = pd.concat([lr_metric, rg1_metric, rg2_metric, ls1_metric, ls2_metric], axis = 1)

final_metric

In [ ]:
# Among these , the model Ridge1 looks optimal as R2 is more stable and RSS/MSE slightly smaller, 
# otherwise Ridge1 and Lasso1 are comparable
#Also doubling alpha slightly reduces effectiveness of regularization

In [ ]:
# Build a dataframe for instant inght into regularization shrinkage and comparison
betas = pd.DataFrame(index=x_train_6.columns)
betas.rows = x_train_6.columns

In [ ]:
betas['Linear'] = reg.coef_
betas['Ridge1'] = ridge1.coef_
betas['Lasso1'] = lasso1.coef_
betas['Ridge2'] = ridge2.coef_
betas['Lasso2'] = lasso2.coef_

In [ ]:
betas.sort_values(by='Ridge1', ascending = False)

In [ ]:
# Print intercept
print('Linear  ' , reg.intercept_)
print('Ridge1  ' , ridge1.intercept_)
print('Lasso1  ' , lasso1.intercept_)
print('Ridge2  ' , ridge2.intercept_)
print('Lasso2  ' , lasso2.intercept_)

In [ ]:
# Our choice is  Ridge (model named Ridge 1) however for questions build another model 
# Its possible that this might overfit slightly, lets see

In [ ]:
X_train.head()

In [ ]:
params = {'alpha': [0.000008,0.0001,0.0006, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}
# cross validation
lasso_all = Lasso()
lasso_all_model_cv_explained_variance = runRegularization(lasso_all,'explained_variance',5)
    
lasso_all_model_cv_explained_variance.fit(X_train, y_train)      

In [ ]:
lasso_all = Lasso(alpha=lasso_all_model_cv_explained_variance.best_params_['alpha'])
lasso_all.fit(X_train, y_train)

In [ ]:
x_all = df_validation
y_pred_train = lasso_all.predict(X_train)
y_pred_validation = lasso_all.predict(x_all)

metricLasso_all = model_effectiveness(y_train,y_pred_train,y_validation,y_pred_validation,explained_variance_score(y_train,y_pred_train))

In [ ]:
betas_all = pd.DataFrame(index=X_train.columns)
betas_all.rows = X_train.columns
betas_all['Lasso_all'] = lasso_all.coef_
betas_all.sort_values(by='Lasso_all', ascending = False)

In [ ]:
X_train_new = X_train.drop(['GrLivArea','OverallQual','TotalBsmtSF','MasVnrArea','GarageArea'],axis =1)

In [ ]:
params = {'alpha': [0.000008,0.0001,0.0006, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}
# cross validation
lasso_all_drop_top5 = Lasso()
lasso_all_drop_top5_model_cv_explained_variance = runRegularization(lasso_all_drop_top5,'explained_variance',5)
    
lasso_all_drop_top5_model_cv_explained_variance.fit(X_train_new, y_train) 

In [ ]:
lasso_all_drop_top5 = Lasso(alpha=lasso_all_drop_top5_model_cv_explained_variance.best_params_['alpha'])
lasso_all_drop_top5.fit(X_train_new, y_train)

In [ ]:
x_all_drop_top5 = df_validation.drop(['GrLivArea','OverallQual','TotalBsmtSF','MasVnrArea','GarageArea'],axis =1)
y_pred_train = lasso_all_drop_top5.predict(X_train_new)
y_pred_validation = lasso_all_drop_top5.predict(x_all_drop_top5)

metricLasso_all_drop_top5 = model_effectiveness(y_train,y_pred_train,y_validation,y_pred_validation,explained_variance_score(y_train,y_pred_train))

In [ ]:
betas_all_drop_top5 = pd.DataFrame(index=X_train_new.columns)
betas_all_drop_top5.rows = X_train_new.columns

In [ ]:
betas_all_drop_top5['Lasso_all_drop_top5'] = lasso_all_drop_top5.coef_

In [ ]:
betas_all_drop_top5.sort_values(by='Lasso_all_drop_top5', ascending = False)

In [ ]:
print("End of Regression , thanks Upgrade and  IITB for nice assignment, it was good learning")